Partie 1
#Chargement des données 

In [3]:
import pandas as pd
import re

def extract_data(input_path, pattern):
    with open(input_path, 'r') as file:
        content = file.read()
    matches = re.findall(pattern, content, re.MULTILINE)
    data = []
    for match in matches:
        numbers = match[1].replace('\n', ' ').split()
        numbers_float = [float(number) for number in numbers]
        data.append(numbers_float)
    return pd.DataFrame(data)

rocks_path = 'data/sonar.rocks'
mines_path = 'data/sonar.mines'

# Patterns for extracting train and test data
train_pattern = r'\*(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'
test_pattern = r'^(?!\*)(CR|CM)\d+:\n\{([\d\s\.\n]+)\}'

# Extracting train and test data
rocks_train_df = extract_data(rocks_path, train_pattern)
mines_train_df = extract_data(mines_path, train_pattern)
rocks_train_df['Label'] = 'R'  
mines_train_df['Label'] = 'M' 
train_df = pd.concat([rocks_train_df, mines_train_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

rocks_test_df = extract_data(rocks_path, test_pattern)
mines_test_df = extract_data(mines_path, test_pattern)
test_df = pd.concat([rocks_test_df, mines_test_df], ignore_index=True).sample(frac=1).reset_index(drop=True)

print(len(train_df))
print(len(test_df)) 


104
104


Partie 2

##Apprentissage sur train

In [4]:
import numpy as np

def perceptron_batch(training_features, training_labels, epochs=100, learning_rate=0.7):
    """
    Entraîne un perceptron en utilisant l'algorithme batch.

    :param training_features: numpy.ndarray, les caractéristiques d'entraînement.
    :param training_labels: numpy.ndarray, les étiquettes d'entraînement (doivent être -1 ou 1).
    :param threshold: int, le nombre d'itérations pour l'entraînement.
    :param learning_rate: float, le taux d'apprentissage.
    :return: numpy.ndarray, les poids appris.
    """
    # Initialisation des poids
    weights = np.zeros(training_features.shape[1] + 1)
    
    # Entraînement
    for _ in range(epochs):
        # Initialisation du vecteur de mise à jour pour cette époque
        update = np.zeros(training_features.shape[1] + 1)
        
        for inputs, label in zip(training_features, training_labels):
            # Calcul de la prédiction
            activation = np.dot(inputs, weights[1:]) + weights[0]
            if activation >= 0:
                prediction = 1
            else:
                prediction = -1
            
            # Accumulation de la mise à jour basée sur l'erreur
            update[1:] += learning_rate * (label - prediction) * inputs
            update[0] += learning_rate * (label - prediction)
        
        # Mise à jour des poids à la fin de l'époque
        weights += update
    
    return weights

# Préparation des données
# Supposons que train_df est déjà défini et mélangé
# Convertir les étiquettes 'R' et 'M' en valeurs numériques -1 et 1
train_df['Label'] = train_df['Label'].apply(lambda x: 1 if x == 'M' else -1)

# Séparation des caractéristiques et des étiquettes
features = train_df.iloc[:, :-1].values
labels = train_df.iloc[:, -1].values

# Entraînement du perceptron
weights = perceptron_batch(features, labels)

print("Poids appris du perceptron :")
print(weights)

Poids appris du perceptron :
[-116.2       32.73032   56.93142   51.60106   76.34466   83.69928
   50.61378   27.61612  -10.66646   85.98506  102.11068  141.42002
  155.35254  134.55974   65.70396  -22.82    -138.24972 -117.2045
  -72.93384   24.23974    9.70172   22.50332   28.57568   -9.5305
    6.5275   -40.56724  -74.24816  -23.00452   56.19572   88.61762
   35.8351   -83.37392   -8.53496  -65.34444 -129.7128  -194.80048
 -262.8052  -184.44664   23.93454   49.70812  -62.98684   40.63556
   84.64708   56.87738  106.10152  183.85794  155.5834   131.257
  108.21384   73.00944    7.08554   12.43382   11.20378    4.53544
    8.10166    1.6093     1.89154   -0.30814    6.70642    5.68862
    2.43684]


Test sur l'ensemble de test